# Making a correlation matrix to better understand the datasets

In [1]:
# Correlation analysis across MiNDAT.csv and MiNDAT_UNK.csv
# - Computes a combined correlation matrix across both files (numeric columns)
# - Shows correlation of each MiNDAT.csv column with CORRUCYSTIC_DENSITY


import pandas as pd

# Helpful display options (optional)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

# 1) Load the two datasets

# Adjust the paths if your CSVs live elsewhere
path_mindat = "MiNDAT.csv"
path_mindat_unk = "MiNDAT_UNK.csv"

# Read with standard CSV parser; pandas will handle quoted fields and empty cells as NaN
df_mindat = pd.read_csv(path_mindat)
df_mindat_unk = pd.read_csv(path_mindat_unk)

# Normalize column names a bit (strip surrounding whitespace)
df_mindat.columns = df_mindat.columns.str.strip()
df_mindat_unk.columns = df_mindat_unk.columns.str.strip()


# 2) Make sure data are numeric where possible
# - Attempt to convert non-numeric columns to numeric (coerce errors to NaN)
# - Keep non-numeric columns as-is for reference


def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out


df_mindat_num = coerce_numeric(df_mindat)
df_mindat_unk_num = coerce_numeric(df_mindat_unk)

# 3) Build a combined dataset (row-wise concat by column name) and compute Pearson correlation matrix

combined_num = pd.concat(
    [df_mindat_num, df_mindat_unk_num], ignore_index=True, sort=False
)

# Only numeric columns participate in correlation
corr_all = combined_num.corr(numeric_only=True, method="pearson")

print("Combined correlation matrix shape:", corr_all.shape)
corr_all.head(10)

# 4) Relationship of MiNDAT.csv columns to CORRUCYSTIC_DENSITY
# - Compute Pearson correlation between every column in MiNDAT.csv and CORRUCYSTIC_DENSITY
# - Include a simple direction label (positive vs negative)

target_col = "CORRUCYSTIC_DENSITY"
if target_col not in df_mindat_num.columns:
    raise KeyError(f"Column {target_col!r} not found in MiNDAT.csv")

corr_to_target = df_mindat_num.corr(numeric_only=True, method="pearson")[
    target_col
].dropna()

# Build a small summary table with direction
rel_to_target = (
    pd.DataFrame({"corr": corr_to_target})
    .assign(abs_corr=lambda d: d["corr"].abs())
    .assign(
        direction=lambda d: np.where(
            d["corr"] >= 0, "positive (direct)", "negative (inverse)"
        )
    )
    .sort_values("abs_corr", ascending=False)
)

print(
    f"Number of numeric columns correlated with {target_col}: {rel_to_target.shape[0]}"
)
rel_to_target.head(25)  # show the top 25 strongest relationships by absolute value

# 5) (Optional) Save results to CSV for further inspection

corr_all.to_csv("combined_correlation_matrix.csv", index=True)
rel_to_target.to_csv("MiNDAT_vs_CORRUCYSTIC_DENSITY.csv", index=True)

print("Saved:")
print(" - combined_correlation_matrix.csv")
print(" - MiNDAT_vs_CORRUCYSTIC_DENSITY.csv")

Combined correlation matrix shape: (48, 48)
Number of numeric columns correlated with CORRUCYSTIC_DENSITY: 45
Saved:
 - combined_correlation_matrix.csv
 - MiNDAT_vs_CORRUCYSTIC_DENSITY.csv


# Univariate R² of each MiNDAT column vs CORRUCYSTIC_DENSITY

In [1]:
# R² of each MiNDAT column vs CORRUCYSTIC_DENSITY


import pandas as pd
import numpy as np

# Adjust if needed
path_mindat = "MiNDAT.csv"
target_col = "CORRUCYSTIC_DENSITY"

# Load
df = pd.read_csv(path_mindat)


# Coerce all object columns to numeric where possible (non-convertible -> NaN)
def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == object:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out


df_num = coerce_numeric(df)

if target_col not in df_num.columns:
    raise KeyError(f"Column {target_col!r} not found in {path_mindat}")

# Compute Pearson correlations (numeric only), square them to get univariate R²
corr_series = df_num.corr(numeric_only=True, method="pearson")[target_col]

# Drop the target itself and any non-existent values
corr_series = corr_series.drop(labels=[target_col], errors="ignore").dropna()

# Square to get R²
r2_series = corr_series.pow(2).rename("R2")

# Optionally exclude ID-like columns if they’re numeric
r2_series = r2_series.drop(labels=["LOCAL_IDENTIFIER"], errors="ignore")

# Sort by strongest R²
r2_sorted = r2_series.sort_values(ascending=False)

# Show top results
print(f"Computed R² against '{target_col}' for {r2_sorted.shape[0]} numeric features.")
display(r2_sorted.head(25))

# Save full ranking
out_csv = "MiNDAT_univariate_R2_vs_CORRUCYSTIC_DENSITY.csv"
r2_sorted.to_csv(out_csv, header=True)
print(f"Saved: {out_csv}")

Computed R² against 'CORRUCYSTIC_DENSITY' for 43 numeric features.


T\!          0.000332
b1oRb13      0.000275
~7*          0.000269
jNhEum       0.000236
i]7V         0.000230
.6AvGp       0.000147
!;@Jw        0.000131
Z~x0<k       0.000115
r1Ng         0.000109
|G}          0.000105
3I\y         0.000102
fPqsI        0.000100
>?64:        0.000084
14W$Q        0.000084
]xq          0.000083
ZVf          0.000064
;<"<i(T      0.000062
b2oRb13      0.000059
.o<m         0.000059
U"r          0.000058
PZ8          0.000043
Jv[i         0.000043
w-u:jN'qI    0.000043
^%a;         0.000041
%IiL7w       0.000031
Name: R2, dtype: float64

Saved: MiNDAT_univariate_R2_vs_CORRUCYSTIC_DENSITY.csv
